In [1]:
import numpy as np
import nltk
from nltk.stem import PorterStemmer 
from nltk.tokenize import sent_tokenize, word_tokenize  

from nltk.corpus import stopwords
import re
from collections import defaultdict
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
Location = '/work/Information retrieval/Inputs/AutomationTxt/'
FileNames = ['arcwebAutomation.txt','enterprisersprojectAutomation.txt', 'RedHatAutomation.txt', 'sganalyticsAutomation.txt', 'thehindubusinesslineAutomation.txt', 'ukessaysAutomation.txt', 'vmwareAutomation.txt']

In [3]:
FileLoc = [Location+i for i in FileNames]

In [5]:
ps = PorterStemmer()
Lemm = WordNetLemmatizer()
stop_words = stopwords.words('english')

In [10]:
def preprocess(text,Stopword):
    text = text.lower()
    pattern = r'^\s*|\s\s*'
    pat = r'[^a-zA-z0-9.,\s]'
    temp = re.sub(pattern, ' ', text).strip()
    CleanTxt = re.sub(pat, '', temp).strip()
    TokenTxt = [words if words not in Stopword else '' for words in CleanTxt.split(' ')]
    return TokenTxt

def inverted_index(stop_words = stop_words,file_names = FileLoc):
    dictionary = {}
    documents = {}
    for i in range(len(file_names)):
        doc_no = i
        with open(file_names[i], 'r',  errors ="surrogateescape") as f:
            s = f.read()
            key = 'Automation_' + str(doc_no)
            documents.setdefault(key,[])
            documents[key].append(s)
            #Change lowercase and removing stopwords, SpecialChar, & Tokenize 
            s = preprocess(s,stop_words)
            doc = []
            doc = list(filter(None, s)) 
            Lemmetzd = []
            for i in doc:
                Lemmetzd.append(Lemm.lemmatize(i))
                #creating posting list
                for x in Lemmetzd:
                    key = x
                    dictionary.setdefault(key, [])
                    dictionary[key].append(doc_no)
                    #removing duplicates
                    dictionary = {a:list(set(b)) for a, b in dictionary.items()}
    print (dictionary)
    return dictionary,documents

def positional_index(stop_words = stop_words,file_names = FileLoc):
    dictionary = {}
    documents = {}
    for i in range(len(file_names)):
        doc_no = i
        with open(file_names[i], 'r', errors ="surrogateescape") as f:
            s = f.read() 
        key = 'Automation_' + str(doc_no)
        documents.setdefault(key,[])
        documents[key].append(s)
        #Change lowercase and removing stopwords, SpecialChar, & Tokenize 
        s = preprocess(s,stop_words)
        doc = []
        doc = list(filter(None, s)) 
        temp_dict = {}
        Lemmetzd = []
        #Lemmatizing
        for i in doc:
            Lemmetzd.append(Lemm.lemmatize(i))
        #creating positional index posting lists
        a = 0
        for x in Lemmetzd:
            key = x
            temp_dict.setdefault(key, [])
            temp_dict[key].append(a)
            a += 1

        for x in temp_dict:
            if dictionary.get(x):
                dictionary[x][doc_no] = temp_dict.get(x)
            else:
                key = x
                dictionary.setdefault(key, [])
                dictionary[key] = {}
                dictionary[x][doc_no] = temp_dict.get(x)
    return dictionary,documents


def postfix(infix_tokens):
     
    #precendence initialization
    precedence = {}
    precedence['NOT'] = 3
    precedence['AND'] = 2
    precedence['OR'] = 1
    precedence['('] = 0
    precedence[')'] = 0
    
    output = []
    operator_stack = []
    
    #creating postfix expression
    for token in infix_tokens:
        if (token == '('):
            print('Stack',operator_stack)
            #print('Stack',operator)
            operator_stack.append(token)
        elif (token == ')'):
            operator = operator_stack.pop()
            print('Stack',operator_stack)
            print('Stack',operator)
            while operator != '(':
                output.append(operator)
                operator = operator_stack.pop()
                print('Stack',operator_stack)
                print('Stack',operator)
        elif (token in precedence):
            if (operator_stack):
                current_operator = operator_stack[-1]
                while (operator_stack and precedence[current_operator] > precedence[token]):
                    output.append(operator_stack.pop())
                    if (operator_stack):
                        current_operator = operator_stack[-1]
            operator_stack.append(token)
        else:
            output.append(token.lower())

 

    #while staack is not empty appending
    while (operator_stack):
        output.append(operator_stack.pop())
        
    return output

#AND
def AND_op(word1,word2):
    if ((word1) and (word2)):
        return set(word1).intersection(word2)
    else:
        return set()
 
#OR
def OR_op(word1,word2):
    return set(word1).union(word2)
 
#NOT
def NOT_op(a):
    tot_docs = list(range(0,56))
    return set(tot_docs).symmetric_difference(a)

def process_query (q, dictionary_inverted):
    q = q.replace('(', '( ')
    q = q.replace(')', ' )')
    q = q.split(' ')
    query = []
    for i in q:
        query.append(i)
    for i in range(0,len(query)):
        if ( query[i]== 'and' or query[i]== 'or' or query[i]== 'not'):
            query[i] = query[i].upper()
    results_stack = []
    postfix_queue = postfix(query)

    #evaluating postfix query expression
    for i in postfix_queue:
        if ( i!= 'AND' and i!= 'OR' and i!= 'NOT'):
            i = i.replace('(', ' ')
            i = i.replace(')', ' ')
            i = i.lower()
            i = dictionary_inverted.get(i)
            results_stack.append(i)
        elif (i=='AND'):
            a = results_stack.pop()
            b = results_stack.pop()
            results_stack.append(AND_op(a,b))
        elif (i=='OR'):
            a = results_stack.pop()
            b = results_stack.pop()
            results_stack.append(OR_op(a,b))
        elif (i == 'NOT'):
            a = results_stack.pop()
            #print(a)
            results_stack.append(NOT_op(a))

    return results_stack.pop()

In [11]:
index, doc = inverted_index([])
print('Printing Inverted index \n -------------------------------------------------------------------------------')
print(index)
pidic, pidoc = positional_index([])
print('Printing Positional index \n -------------------------------------------------------------------------------')
print(pidic)

{'what': [0, 1, 2, 4, 6], 'is': [0, 1, 2, 3, 4, 5, 6], 'behind': [0, 2], 'the': [0, 1, 2, 3, 4, 5, 6], 'growth': [0, 1, 3, 4, 5], 'of': [0, 1, 2, 3, 4, 5, 6], 'automation': [0, 1, 2, 3, 4, 5, 6], 'and': [0, 1, 2, 3, 4, 5, 6], 'it': [0, 1, 2, 3, 4, 5, 6], 'in': [0, 1, 2, 3, 4, 5, 6], 'india': [0, 3, 4], 'overview': [0], 'arc': [0], 'advisory': [0, 4], 'group': [0, 1, 2], 'recent': [0, 4, 6], 'forum': [0, 3], 'bangalore': [0], 'focused': [0, 2], 'on': [0, 1, 2, 3, 4, 5, 6], 'latest': [0], 'trend': [0, 1, 3, 4, 6], 'innovative': [0], 'technology': [0, 1, 3, 4, 5, 6], 'it,': [0], 'automation,': [0, 1, 2, 4, 5, 6], 'indiaand': [0], 'enterprise': [0, 1, 2, 4, 5, 6], 'solutions,': [0], 'including': [0], 'related': [0, 1, 6], 'opportunity': [0, 1, 2, 3, 4], 'challenge': [0, 4, 5], 'facing': [0], 'various': [0, 1], 'industries.': [0], 'this': [0, 1, 2, 3, 4, 5, 6], 'insight': [0], 'focus': [0, 3, 4, 5, 6], 'presentation': [0], 'by': [0, 1, 2, 3, 4, 5, 6], 'achyuta': [0], 'ghosh,': [0], 'head': 

In [12]:
process_query("(automation and cloud) or (process and tasks)", index)

Stack []
Stack ['(']
Stack AND
Stack []
Stack (
Stack ['OR']
Stack ['OR', '(']
Stack AND
Stack ['OR']
Stack (


{2, 6}

In [14]:
process_query("(automation and cloud)", index)

Stack []
Stack ['(']
Stack AND
Stack []
Stack (


{2, 6}

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2bbb1852-54a2-4cb7-b571-37abc5ed36bd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>